In [0]:
#fazer a leitura sequencial dos IDs dos deputados 
#fazer loop para chamada da api despesas
#gravar um json por deputado no diretorio
from pyspark.sql import functions as F
import requests
import json
from datetime import datetime
import time

# Caminho do seu arquivo JSON
caminho_json = "dbfs:/Volumes/workspace/deputados/deputados/bronze/deputados_*.json"

# Lê o arquivo JSON
df = spark.read.option("multiline", "true").json(caminho_json)

# Extrai apenas os IDs do array 'dados'
df_ids = df.select(
    F.explode("dados").alias("deputado")
).select(
    F.col("deputado.id").alias("id")
)

# Converte os IDs para uma lista Python
ids_deputados = [row['id'] for row in df_ids.collect()]
print(f"Total de deputados para buscar: {len(ids_deputados)}")

# Pasta de destino para salvar as despesas
pasta_destino = "/Volumes/workspace/deputados/deputados/despesas"

# Garante que a pasta existe
dbutils.fs.mkdirs(pasta_destino)

def buscar_despesas_com_paginacao(id_deputado, ano=2025):
    """
    Busca despesas com tratamento de paginação
    """
    url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{id_deputado}/despesas"
    
    todas_despesas = []
    pagina = 1
    
    try:
        while True:
            params = {
                'ano': ano,
                'ordem': 'ASC',
                'ordenarPor': 'ano',
                'itens': 100,
                'pagina': pagina
            }
            
            response = requests.get(url, params=params)
            response.raise_for_status()
            
            dados = response.json()
            despesas_pagina = dados.get('dados', [])
            
            if not despesas_pagina:
                break
                
            todas_despesas.extend(despesas_pagina)
            print(f"  Página {pagina}: {len(despesas_pagina)} despesas")
            
            # Verifica se há mais páginas
            links = dados.get('links', [])
            tem_proxima = any(link.get('rel') == 'next' for link in links)
            
            if not tem_proxima:
                break
                
            pagina += 1
            time.sleep(0.2)  # Delay entre páginas
            
        return {
            'idDeputado': id_deputado,
            'dados': todas_despesas,
            'totalDespesas': len(todas_despesas),
            'totalPaginas': pagina - 1,
            'dataExtracao': datetime.now().isoformat()
        }
        
    except requests.exceptions.RequestException as e:
        print(f"Erro ao buscar despesas do deputado {id_deputado}: {e}")
        return None
    except Exception as e:
        print(f"Erro inesperado com deputado {id_deputado}: {e}")
        return None

# Lista para armazenar resultados
todas_despesas = []
deputados_com_erro = []

# Loop através de todos os IDs
for i, id_deputado in enumerate(ids_deputados):
    print(f"\n--- Processando deputado {i+1}/{len(ids_deputados)} - ID: {id_deputado} ---")
    
    # Busca as despesas com paginação
    despesas = buscar_despesas_com_paginacao(id_deputado)
    
    if despesas:
        print(f"✅ Total encontrado: {despesas['totalDespesas']} despesas em {despesas['totalPaginas']} páginas")
        todas_despesas.append(despesas)
        
        # Salva individualmente
        nome_arquivo = f"despesas_{id_deputado}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        caminho_individual = f"{pasta_destino}/individual/{nome_arquivo}"
        
        dbutils.fs.mkdirs(f"{pasta_destino}/individual")
        dbutils.fs.put(
            caminho_individual,
            json.dumps(despesas, ensure_ascii=False, indent=2),
            overwrite=True
        )
    else:
        print(f"❌ Erro ao processar deputado {id_deputado}")
        deputados_com_erro.append(id_deputado)
    
    # Delay entre deputados
    time.sleep(0.5)


In [0]:
# Salva todas as despesas consolidadas em um único arquivo
if todas_despesas:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    caminho_consolidado = f"{pasta_destino}/despesas_consolidadas_{timestamp}.json"
    
    # Calcula estatísticas
    total_despesas = sum([d['totalDespesas'] for d in todas_despesas])
    total_deputados_ok = len(todas_despesas)
    
    dados_consolidados = {
        'metadados': {
            'totalDeputadosProcessados': total_deputados_ok,
            'totalDeputadosComErro': len(deputados_com_erro),
            'totalDespesasColetadas': total_despesas,
            'dataExtracao': datetime.now().isoformat(),
            'anoReferencia': 2025,
            'deputadosComErro': deputados_com_erro
        },
        'despesas': todas_despesas
    }
    
    # Salva o arquivo consolidado com dbutils
    json_consolidado = json.dumps(dados_consolidados, ensure_ascii=False, indent=2)
    dbutils.fs.put(
        caminho_consolidado,
        json_consolidado,
        overwrite=True
    )
    
    print(f"\n✅ Processamento concluído!")
    print(f"✅ Deputados processados com sucesso: {total_deputados_ok}")
    print(f"✅ Deputados com erro: {len(deputados_com_erro)}")
    print(f"✅ Total de despesas coletadas: {total_despesas}")
    print(f"✅ Arquivo consolidado salvo em: {caminho_consolidado}")
    
    if deputados_com_erro:
        print(f"❌ IDs com erro: {deputados_com_erro}")
    
else:
    print("❌ Nenhuma despesa foi encontrada")